In [ ]:
import sys, time
import math
import numpy as np
from pykern import pkcli
from pykern.pkcollections import PKDict

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import element

import scipy.constants as const

In [ ]:
# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

# 3D plotting
import plotly.graph_objects as go

# Specify which plots are desired
RENDER_2D_PLOTS = True
RENDER_3D_PLOTS = True

In [ ]:
"""CalcIntFromElecField(_arI, _inWfr, _inPol, _inIntType, _inDepType, _inE, _inX, _inY)
function calculates/"extracts" Intensity from pre-calculated Electric Field
:param _arI: output resulting Intensity array (should be allocated in Python script before calling this function)
:param _inWfr: input pre-calculated Wavefront structure (instance of SRWLWfr)
:param _inPol: input switch specifying polarization component to be extracted:
               =0 -Linear Horizontal; 
               =1 -Linear Vertical; 
               =2 -Linear 45 degrees; 
               =3 -Linear 135 degrees;
               =4 -Circular Right; 
               =5 -Circular Left; 
               =6 -Total
:param _inIntType: input switch specifying "type" of a characteristic to be extracted:
               =0 -"Single-Electron" Intensity; 
               =1 -"Multi-Electron" Intensity; 
               =2 -"Single-Electron" Flux; 
               =3 -"Multi-Electron" Flux; 
               =4 -"Single-Electron" Radiation Phase; NOTE: requires DOUBLE PRECISION array via srwlib.array('d', ...)
               =5 -Re(E): Real part of Single-Electron Electric Field;
               =6 -Im(E): Imaginary part of Single-Electron Electric Field;
               =7 -"Single-Electron" Intensity, integrated over Time or Photon Energy (i.e. Fluence)
               =8 -"Single-Electron" Mutual Intensity (i.e. E(r)E*(r')) 
:param _inDepType: input switch specifying type of dependence to be extracted:
               =0 -vs e (photon energy or time);
               =1 -vs x (horizontal position or angle);
               =2 -vs y (vertical position or angle);
               =3 -vs x&y (horizontal and vertical positions or angles);
               =4 -vs e&x (photon energy or time and horizontal position or angle);
               =5 -vs e&y (photon energy or time and vertical position or angle);
               =6 -vs e&x&y (photon energy or time, horizontal and vertical positions or angles);
:param _inE: input photon energy [eV] or time [s] to keep fixed (to be taken into account for dependences vs x, y, x&y)
:param _inX: input horizontal position [m] to keep fixed (to be taken into account for dependences vs e, y, e&y)
:param _inY: input vertical position [m] to keep fixed (to be taken into account for dependences vs e, x, e&x)
"""
import srwlib
from srwlib import srwl

In [ ]:
# define a simple lattice
dr1 = element.Drift(0.2)                      # 20 cm drift
crystal = element.CrystalSlice(PKDict(length=0.1))  # 10 cm single-slice crystal 
dr2 = element.Drift(0.02)                     #  2 cm drift
lattice = [(dr1,'default'), (crystal,'abcd'), (dr2,'default')]
  
current_position = 0.0 
print('Initial z position of the pulse:', current_position, 'm')

In [ ]:
# specify parameters
_LASER_PULSE_SLICE_DEFAULTS = PKDict(
    sigrW=0.000186,
    propLen=15,
    pulseE=0.001,
    poltype=1,
    sampFact=1,
    numsig=3.,
    mx=0,
    my=0,
)
_LASER_PULSE_DEFAULTS = PKDict(
        phE=1.55,
        nslice=1,
        chirp=0,
        w0=128.9e-6,
        a0=.01,
        dw0x=0.0,
        dw0y=0.0,
        z_waist=0.,
        dzwx=0.0,
        dzwy=0.0,
        tau_fwhm=2.e-10,
        z_center=0.,
        x_shift = 0.,
        y_shift=0.,
        d_to_w=0.009,
        slice_params=_LASER_PULSE_SLICE_DEFAULTS,
)

In [ ]:
# Override some of the default parameters
# First, instantiate the default parameters:
params = _LASER_PULSE_DEFAULTS.copy()

lambda0 = 8.e-7                     # wavelength [m]
phE = const.h * const.c / lambda0   # photon energy [J]
phE_ev = phE / const.e              # photon energy [eV]
print(' photon energy = {0:4.2E} [J]'.format(phE) + ' = {0:4.2f} [eV]'.format(phE_ev))

print(' params.tau_fwhm = {0:4.2E} [s]'.format(params.tau_fwhm))
tau_fwhm = 0.3e-13                  # FWHM pulse length [s]
params.tau_fwhm = tau_fwhm

# params.d_to_w = 0.0     #  [m] distance from the initial pulse location to the loc-n of the beam waist, > 0 if converging 
params.d_to_w = 0.011     #  [m] distance from the initial pulse location to the loc-n of the beam waist, > 0 if converging 

In [ ]:
# Instantiate the laser pulse
thisPulse = pulse.LaserPulse(params)

In [ ]:
# initial pulse - intensity

# choose one of the laser pulse slices, and grab its SRW wavefront object
slice_array=thisPulse.slice
slice_number = 0
wfr0=slice_array[slice_number].wfr

intensity0 = srwlib.array('f', [0]*wfr0.mesh.nx*wfr0.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(intensity0, wfr0, 0, 0, 3, wfr0.mesh.eStart, 0, 0) #extracts intensity

##Reshaping electric field data from flat to 2D array
intens_2d_0 = np.array(intensity0).reshape((wfr0.mesh.nx, wfr0.mesh.ny), order='C')
wfrsizei=np.size(intensity0)

print('Size of initial wavefront data array (coordinate):',np.shape(intens_2d_0))
x0=np.linspace(wfr0.mesh.xStart,wfr0.mesh.xFin,wfr0.mesh.nx)
y0=np.linspace(wfr0.mesh.yStart,wfr0.mesh.yFin,wfr0.mesh.ny)

In [ ]:
# plot the computed intensity
if RENDER_2D_PLOTS:
    with plt.style.context(('seaborn-poster')):
        fig = plt.figure(figsize=(12,7))
        ax = fig.gca()

        # By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
        plt.pcolormesh(np.multiply(x0,1e6), np.multiply(y0,1e6), intens_2d_0, cmap=plt.cm.viridis,shading='auto')
        plt.colorbar()
        ax.set_ylabel(r'Vertical Position [$\mu m$]')
        ax.set_xlabel(r'Horizontal Position [$\mu m$]')
        ax.set_title('Intensity before the crystal')

In [ ]:
# plot the 3D intensity
if RENDER_3D_PLOTS:
    fig = go.Figure(data=[go.Surface(x=x0, y=y0, z=intens_2d_0)])
    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
        highlightcolor="limegreen", project_z=True)
    )
    fig.update_layout(title='Laser pulse intensity before the crystal', autosize=False,
        width=800, height=700, margin=dict(l=65, r=50, b=65, t=90)
    )
    fig.show()

In [ ]:
# initial pulse - phase
wfr1=slice_array[slice_number].wfr

phase1 = srwlib.array('d', [0]*wfr1.mesh.nx*wfr1.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(phase1, wfr1, 0, 4, 3, wfr1.mesh.eStart, 0, 0) #extracts the phase; must use double precision

##Reshaping electric field data from flat to 2D array
phase_2d_1 = np.array(phase1).reshape((wfr1.mesh.nx, wfr1.mesh.ny), order='C')
wfrsize=np.size(phase1)

print('Size of initial wavefront data array (coordinate):',np.shape(phase_2d_1))
x1=np.linspace(wfr1.mesh.xStart,wfr1.mesh.xFin,wfr1.mesh.nx)
y1=np.linspace(wfr1.mesh.yStart,wfr1.mesh.yFin,wfr1.mesh.ny)

In [ ]:
# plot the phase
if RENDER_2D_PLOTS:
    with plt.style.context(('seaborn-poster')):
        fig = plt.figure(figsize=(12,7))
        ax = fig.gca()

        # By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
        plt.pcolormesh(np.multiply(x1,1e6), np.multiply(y1,1e6), phase_2d_1, cmap=plt.cm.viridis,shading='auto')
        plt.colorbar()
        ax.set_ylabel(r'Vertical Position [$\mu m$]')
        ax.set_xlabel(r'Horizontal Position [$\mu m$]')
        ax.set_title('Phase before the crystal')
    
    phi_diff = np.max(phase_2d_1) - np.min(phase_2d_1)
    phi_rel_diff = 100. * phi_diff / math.pi / 2.
    print(' ')
    print(' phase_diff = {0:4.3f} [rad]'.format(phi_diff) + ' = {0:4.3f} %'.format(phi_rel_diff))
    print(' ')

In [ ]:
# plot the 3D phase information
if RENDER_3D_PLOTS:
    fig = go.Figure(data=[go.Surface(x=x1, y=y1, z=phase_2d_1)])
    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
        highlightcolor="limegreen", project_z=True)
    )
    fig.update_layout(title='Wavefront phases before the crystal', autosize=False,
        width=800, height=700, margin=dict(l=65, r=50, b=65, t=90)
    )
    fig.show()

In [ ]:
# thisPulse = pulse.LaserPulse(params)
#  Propagate the pulse through the optical beamline:
for i in lattice:
    current_elem, prop_type = i 
    # print(current_elem, prop_type) 
    thisPulse = current_elem.propagate(thisPulse, prop_type)
    current_position += current_elem.length
    print('Current position in the beamline:', current_position, ' m')

In [ ]:
# transmitted pulse - intensity
wfr2=slice_array[slice_number].wfr

intensity2 = srwlib.array('f', [0]*wfr2.mesh.nx*wfr2.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(intensity2, wfr2, 0, 0, 3, wfr0.mesh.eStart, 0, 0) #extracts intensity

##Reshaping electric field data from flat to 2D array
intens_2d_2 = np.array(intensity2).reshape((wfr2.mesh.nx, wfr2.mesh.ny), order='C')
wfrsizei=np.size(intensity2)

print('Size of initial wavefront data array (coordinate):',np.shape(intens_2d_2))
x2=np.linspace(wfr2.mesh.xStart,wfr2.mesh.xFin,wfr2.mesh.nx)
y2=np.linspace(wfr2.mesh.yStart,wfr2.mesh.yFin,wfr2.mesh.ny)

In [ ]:
# plot the computed intensity
if RENDER_2D_PLOTS:
    with plt.style.context(('seaborn-poster')):
        fig = plt.figure(figsize=(12,7))
        ax = fig.gca()

        # By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
        plt.pcolormesh(np.multiply(x2,1e6), np.multiply(y2,1e6), intens_2d_2, cmap=plt.cm.viridis,shading='auto')
        plt.colorbar()
        ax.set_ylabel(r'Vertical Position [$\mu m$]')
        ax.set_xlabel(r'Horizontal Position [$\mu m$]')
        ax.set_title('Intensity after the crystal')

In [ ]:
# plot the 3D intensity
if RENDER_3D_PLOTS:
    fig = go.Figure(data=[go.Surface(x=x2, y=y2, z=intens_2d_2)])
    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
        highlightcolor="limegreen", project_z=True)
    )
    fig.update_layout(title='Laser pulse intensity after the crystal', autosize=False,
        width=800, height=700, margin=dict(l=65, r=50, b=65, t=90)
    )
    fig.show()

In [ ]:
# transmitted pulse - phase
wfr3=slice_array[slice_number].wfr

phase3 = srwlib.array('d', [0]*wfr3.mesh.nx*wfr3.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(phase3, wfr3, 0, 4, 3, wfr3.mesh.eStart, 0, 0) #extracts the phase; must use double precision

##Reshaping electric field data from flat to 2D array
phase_2d_3 = np.array(phase3).reshape((wfr3.mesh.nx, wfr3.mesh.ny), order='C')
wfrsize=np.size(phase3)

print('Size of initial wavefront data array (coordinate):',np.shape(phase_2d_3))
x3=np.linspace(wfr3.mesh.xStart,wfr3.mesh.xFin,wfr3.mesh.nx)
y3=np.linspace(wfr3.mesh.yStart,wfr3.mesh.yFin,wfr3.mesh.ny)

In [ ]:
# plot the phase
if RENDER_2D_PLOTS:
    with plt.style.context(('seaborn-poster')):
        fig = plt.figure(figsize=(12,7))
        ax = fig.gca()

        # By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
        plt.pcolormesh(np.multiply(x3,1e6), np.multiply(y3,1e6), phase_2d_3, cmap=plt.cm.viridis,shading='auto')
        plt.colorbar()
        ax.set_ylabel(r'Vertical Position [$\mu m$]')
        ax.set_xlabel(r'Horizontal Position [$\mu m$]')
        ax.set_title('Phase after the crystal')
    
    phi_diff = np.max(phase_2d_3) - np.min(phase_2d_3)
    phi_rel_diff = 100. * phi_diff / math.pi / 2.
    print(' ')
    print(' phase_diff = {0:4.3f} [rad]'.format(phi_diff) + ' = {0:4.3f} %'.format(phi_rel_diff))
    print(' ')

In [ ]:
# plot the 3D phase information
if RENDER_3D_PLOTS:
    fig = go.Figure(data=[go.Surface(x=x3, y=y3, z=phase_2d_3)])
    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
        highlightcolor="limegreen", project_z=True)
    )
    fig.update_layout(title='Wavefront phases after the crystal', autosize=False,
        width=800, height=700, margin=dict(l=65, r=50, b=65, t=90)
    )
    fig.show()